
# DSC Basics, Part II

This is a continuation of [DSC Basics, Part I](Intro_Syntax_I.html). Here we continue the example to illustrate new concepts and syntax, particularly the use of parameters to create multiple similar modules.

Material used in this example are in [the DSC vignettes repo](https://github.com/stephenslab/dsc/tree/master/vignettes/one_sample_location).

## Defining similar modules via parameters

Sometimes it can be helpful to specify multiple similar modules.

For example, our `normal` module simulated 100 observations from an $N(0,1)$ distribution. What if we would like to also define a module that simulates 1000 observations from $N(0,1)$?

DSC makes this easy by:

1. Allowing module definitions to include parameter values to be passed to the script.

2. Allowing parameters to be given more than one value, separated by ",", each defining a different module.

### Example 1: a single parameter

For example, consider the following DSC syntax:

```
normal: R( x = rnorm(n, 0, 1) )
  n: 100, 1000
  $data: x
  $true_mean: 0
```

This introduces two new pieces of syntax:

1. `n` is a parameter, whose value is passed to the script variable `n`. (The parameter and script variable are conceptually distinct, even though they have the same name. By default DSC passes parameters to the script variable with the same name, but this behaviour can be modified by use of [@ALIAS](../reference/DSC_Configuration.html#Decorator-ALIAS-122).)

2. The `n: 100, 1000` line defines *two* modules, one with a parameter `n=100` that simulates 100 values, and one with `n=1000` that simulates 1000 observations. Both modules have the same name -- `normal` -- but they are differentiated by the value of the parameter `n`. For example, when querying the results of a dsc we will be able to subset results on the value of `n`. Put another way, `normal` now becomes a "group" containing two similar modules.

<!-- For shorthand we might write `normal(n=100)` and `normal(n=1000)` to refer to these two modules (this is just a convenient shorthand to help our explanation, not a part of DSC syntax). Both modules take the same kind of input (no input) and produce the same type of output (pipeline variables `$data` and `$true_mean`). -->

*NOTE:* We need to explain **order of evaluation** here.

### Example 2: multiple parameters

If you provide more than one value for multiple parameters, DSC considers all combinations. For example:

```
t: R( x = rt(n, df) )
  n: 100, 1000
  df: 2, 4, 10
  $data: x
  $true_mean: 0
```

defines a group of six modules, which simulate 100 or 1000 observations from a $t$ distributions with 2, 4 or 10 degrees of freedom.


### Example 3: inputs and parameters

Now consider the following DSC syntax:

```
trim_mean: R( y = mean(x, trim) )
  trim: 0, 0.2
  x: $data
  $est_mean: y
```

defines two modules which compute the trimmed mean of the pipeline variable
`$data`, with different levels of trimming (0 or 0.2), and output
the result as the pipeline variable `$est_mean`.

Schematically we might represent this as:

```
# trim_mean(trim=0): $data -> $est_mean
# trim_mean(trim=0.2): $data -> $est_mean
```

to indicate we have two different modules, each of which take input
`$data` and output `$est_mean`.

Note: it can be important, conceptually, to distinguish module inputs from module parameters. But in some cases this distinction may seem a gray area. There is a simple rule in DSC: parameters cannot be assigned to be pipeline variables. So in this example, `trim` is a parameter, but `x` is not.

Because this rule uniquely distinguishes parameters from inputs, the DSC
syntax does not require the user to make the distinction explicitly.
However,  it may aid readability to add comments:

```
trim_mean: R( y = mean(x, trim) )
  # parameters
  trim: 0, 0.2

  # input
  x: $data

  # output
  $est_mean: y
```